# Manipulando, analisando, filtrando e resumindo dados em R
A linguagem R tem como um de seus grandes destaques o pacote Tidyverse.<br>
Para falar a verdade, o Tidyverse não é um pacote em si, mas sim um conjunto de pacotes.<br>
Dentro dele, encotramos o `dplyr`, que é instaldo automaticamente com o `install.packages("tidyverse")`.
Vamos aqui fazer uma análise do dataset `Consumo_cerveja.xlsx`. <br>
Esse dataset é o que chamamos de uma série temporal: ele tem uma sequência de datas e os valores de certas variáveis nesta sequência.<br>
Vamos dar uma olhada nessas variáveis e ver se descobrimos algo.<br>

Aqui uma [Cheat Sheet do dplyr](https://github.com/rstudio/cheatsheets/blob/master/data-transformation.pdf)

In [1]:
library("dplyr")
library("readxl")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
dataset <- read_excel("Consumo_cerveja.xlsx")

Agora que lemos o nosso arquivo para dentro do ambiente do R, vamos dar uma olhada nas colunas que temos. Para isso vamos usar a função nativa do `R` , a `names()`

In [3]:
names(dataset)

[1] "Data"                        "Temperatura Media (C)"      
[3] "Temperatura Minima (C)"      "Temperatura Maxima (C)"     
[5] "Precipitacao (mm)"           "Final de Semana"            
[7] "Consumo de cerveja (litros)"

Vamos dar uma olhada no nosso dataset então. <br>
Para imprimir as primeiras `n` linhas nós usamos um comando do tipo `dataset[1:n,]`.<br>
A vírgula no final dos colchetes indica que todas as colunas devem ser selecionadas.<br>
Se a ideia fosse selecionar da linha 5 até a 25, das colunas 3 a 5 usariamos o seguinte comando:<br>
`dataset[5:25,3:5]`<br>
Agora se quiséssemos apenas a linha 22 e a 27 das colunas 2 e 5, usariamos:<br>
`dataset[c(22,27),c(2,5)]`

In [4]:
dataset[1:5,]

Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros)
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2015-01-01,27.30,23.9,32.5,0.0,0,25461
2015-01-02,27.02,24.5,33.5,0.0,0,28972
2015-01-03,24.82,22.4,29.9,0.0,1,30814
2015-01-04,23.98,21.5,28.6,1.2,1,29799
2015-01-05,23.82,21.0,28.3,0.0,0,28900


Podemos ver que embaixo dos nomes das variáveis, temos valores como `<chr>` e `<dbl>`. <br>
Isso indica o tipo da variável interpretada pelo R.<br>
Por exemplo, no caso da data, o R interpretou como `<chr>`, ou seja, um texto. <br>
No caso das temperaturas mínimas e máximas foi interpretado `<dbl>` que significa um número com casas decimais. <br>
No caso da variável 'Fim de Semana', que foi interpretada como numero com casas decimais, é guardado o valor `1` se o dia corresponder a um dia do fim de semana e `0` se for um dia comum. Isso é o que chamamos de uma `variável lógica (binária)`. 
Ou seja, o `R` interpretou de forma errada o tipo da nossa variável, já que não é um numero com casas decimais e sim um valor lógico.
Vamos arrumar isso:

In [5]:
dataset$`Final de Semana` <- as.logical(dataset$`Final de Semana`)
#o `$` é usado para selecionar uma coluna

Vamos ver que, agora, o valor da variável é `<lgl>`, como queríamos.

In [6]:
dataset[1:5,]

Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros)
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>
2015-01-01,27.30,23.9,32.5,0.0,FALSE,25461
2015-01-02,27.02,24.5,33.5,0.0,FALSE,28972
2015-01-03,24.82,22.4,29.9,0.0,TRUE,30814
2015-01-04,23.98,21.5,28.6,1.2,TRUE,29799
2015-01-05,23.82,21.0,28.3,0.0,FALSE,28900


## Deep dive in dplyr

Vamos agora nos aprofundar na biblioteca `dplyr`. <br>
Essa biblioteca é boa porque deixa o nosso código extremamente legível para um ser humano.<br>
Isso ocorre porque ela usa uma estrutura de "verbos" que regem o que será feito com a base de dados.<br>
Os verbos existentes na biblioteca são os seguintes:
1. `mutate()` - que adiciona novas variáveis (colunas) ao dataset a partir de operações/ aplicação de funções nas variáveis existentes 
2. `select()` - seleciona as colunas com base em critérios de nome
3. `filter()` - filtra os dados com base em valores.
4. `summarise()` - reduz um grande número de valores para valores resumo (como média, contagem, desvio, etc). Para quem mexe com excel, aqui seria algo para criar uma tabela dinâmica (pivot table). Em geral é usado com `group_by()`
5. `arrange()`- muda a ordenação das linhas com base em algum critério
<br>
<br>
Todos esses verbos podem ser performados tendo como critério um grupo, usando o `group_by()` , mas é mais comum usar isso no `summarise()`.

A estrutura de utilização dos verbos é bem simples: <br>
`base de dados %>% verbo`

Podemos aplicar vários verbos em seguida também, com a seguinte estutura (a ordem de execução será a ordem desses verbos): <br>
`base de dados %>% verbo1 %>% verbo2 %>% verbo3`

### `mutate()` - adicionando variáveis

Vamos aqui adicionar duas variáveis a partir de funções aplicadas em outras.<br>
Primeiro, vamo classificar dias com precipitação maior que 10mm como dias chuvosos, criando uma coluna lógica. <br>
A segunda coluna criada será a de amplitude térmica, obtida com a diferença entre as temperaturas máximas e mínimas.

In [8]:
dataset %>% mutate(
            `Dia chuvoso` = if_else(`Precipitacao (mm)` >= 10, TRUE, FALSE, missing = NULL),
            # a função if_else tem quatro argumentos. O primeiro é a 'afirmação lógica' a ser avaliada
            # no caso, essa afirmação é verificar se a precipitação é maior que 10mm.
            # o segundo argumento é o que será inputado no caso da afirmação ser verdadeira
            # o terceiro, o que será inputado no caso de falsa
            # e o quarto, o que será inputado no caso de missing value (valor faltante)
            `Amplitude térmica (C)` = `Temperatura Maxima (C)` - `Temperatura Minima (C)`
            )

Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros),Dia chuvoso,Amplitude térmica (C)
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<lgl>,<dbl>
2015-01-01,27.30,23.9,32.5,0.0,FALSE,25461,FALSE,8.6
2015-01-02,27.02,24.5,33.5,0.0,FALSE,28972,FALSE,9.0
2015-01-03,24.82,22.4,29.9,0.0,TRUE,30814,FALSE,7.5
2015-01-04,23.98,21.5,28.6,1.2,TRUE,29799,FALSE,7.1
2015-01-05,23.82,21.0,28.3,0.0,FALSE,28900,FALSE,7.3
2015-01-06,23.78,20.1,30.5,12.2,FALSE,28218,TRUE,10.4
2015-01-07,24.00,19.5,33.7,0.0,FALSE,29732,FALSE,14.2
2015-01-08,24.90,19.5,32.8,48.6,FALSE,28397,TRUE,13.3
2015-01-09,28.20,21.9,34.0,4.4,FALSE,24886,FALSE,12.1


### `select()` - selecionando valores com base em nome

Esse comando é bem simples, nós podemos selecionar as colunas com o critério baseado no nome. 
<br>Isso pode incluir "começa com": `starts_with(padrão procurado)`, "termina com": `ends_with(padrão procurado)`, "contém": `contains(padrão procurado)`... <br>
Pra quem manja mais, pode usar expressões regulares com `matches()` ou muitos outros critérios que vc pode dar uma conferida [aqui!](https://www.rdocumentation.org/packages/tidyselect/versions/1.0.0/topics/select_helpers)

Aqui vamos selecionar todas as colunas que contem a palavra "temperatura" no título e guardar esse resultado na variável "selecting"

In [12]:
selecting <- dataset %>% select(contains("Temperatura"))

In [15]:
selecting[1:5,]

Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C)
<dbl>,<dbl>,<dbl>
27.30,23.9,32.5
27.02,24.5,33.5
24.82,22.4,29.9
23.98,21.5,28.6
23.82,21.0,28.3


### `filter()` - Filtrando nossa base de dados!

Vamos supor que você quer analisar sua base de dados e segmentar essa análise com base em algum critério.. Isso é só um dos cenários em que pode ser muito importante você usar filtros!

Vamos aqui, pegar apenas os dias em que o consumo de cerveja foi acima da média. Para isso vamos usar a função `mean()` nativa do `R`

In [17]:
filtro_cerveja = dataset %>% filter(`Consumo de cerveja (litros)` > mean(`Consumo de cerveja (litros)`, na.rm = TRUE))

In [18]:
filtro_cerveja[1:5,]

Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros)
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>
2015-01-01,27.30,23.9,32.5,0.0,FALSE,25461
2015-01-02,27.02,24.5,33.5,0.0,FALSE,28972
2015-01-03,24.82,22.4,29.9,0.0,TRUE,30814
2015-01-04,23.98,21.5,28.6,1.2,TRUE,29799
2015-01-05,23.82,21.0,28.3,0.0,FALSE,28900


Agora vamos filtrar nossa base, pegando apenas as linhas que correspondem a finais de semana

In [19]:
filtro_fds = dataset %>% filter(`Final de Semana` == TRUE)

In [20]:
filtro_fds[1:5,]

Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros)
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>
2015-01-03,24.82,22.4,29.9,0.0,TRUE,30814
2015-01-04,23.98,21.5,28.6,1.2,TRUE,29799
2015-01-10,26.76,22.1,34.2,0.0,TRUE,37937
2015-01-11,27.62,22.2,34.8,3.4,TRUE,36254
2015-01-17,28.86,22.0,35.8,0.0,TRUE,37690


### `summarise()` - "Tabelas dinâmicas" no R

Usar o verbo `summarise()` vai produzir um único número com a(s) medida(s) resumo escolhidas. <br>
Mas como, muitas vezes, nós queremos segmentar a análise para subgrupos diferentes do dataset, o `summarise()` costuma ser utilizado com o `group_by()`. <br>
O `group_by()` vai fazer com que as medidas resumos sejam feitas com base nos diferentes valores tidos em uma coluna.

Vamos, primeiro, usar o `summarise()` puro. Perceba que ele devolve a medida escolhida levando em consideração todas as linhas.

In [22]:
dataset %>%
  summarise(media = mean(`Consumo de cerveja (litros)`))

media,n
<dbl>,<int>
25401.37,365


Agora, utilizando o `group_by()` podemos segmentar os números com base em uma categoria (se é fim de semana ou não).

In [24]:
dataset %>% 
        group_by(`Final de Semana`) %>%
        summarise(`Média de consumo de cerveja (litros)` = mean(`Consumo de cerveja (litros)`), 
                  `Desvio Padrão no consumo de cerveja (litros)` = sd(`Consumo de cerveja (litros)`))

Final de Semana,Média de consumo de cerveja (litros),Desvio Padrão no consumo de cerveja (litros)
<lgl>,<dbl>,<dbl>
FALSE,23998.22,3787.527
TRUE,28922.72,3830.191


### `arrange()` - Alterando a ordem das linhas do seu dataset

Esse é provavelmente o verbo mais fácil de ser usado. Você passa como argumento um ou mais critérios de ordenação (sendo que a ordem de preferência é a ordem passada). <br>
Caso queira que a ordenação seja decrescente, é só utilizar a função `desc()`.

Ordenando o dataset do maior para o menor consumo de cerveja

In [26]:
desc_consumo_cerveja = dataset %>% arrange(desc(`Consumo de cerveja (litros)`))

In [27]:
desc_consumo_cerveja[1:5,]

Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros)
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>
2015-01-10,26.76,22.1,34.2,0.0,TRUE,37937
2015-01-17,28.86,22.0,35.8,0.0,TRUE,37690
2015-01-11,27.62,22.2,34.8,3.4,TRUE,36254
2015-10-15,28.12,18.9,35.8,0.0,FALSE,35861
2015-10-31,23.18,18.2,29.3,0.0,TRUE,35781


Ordenando o dataset da menor para a maior Temperatura Máxima

In [28]:
ord_max_temp <- dataset %>% arrange(`Temperatura Maxima (C)`)

In [29]:
ord_max_temp[1:5,]

Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros)
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>
2015-07-05,12.90,11.0,14.5,4.4,TRUE,21294
2015-09-12,14.00,12.8,15.6,27.6,TRUE,21814
2015-07-22,14.52,12.8,16.1,0.0,FALSE,14343
2015-09-13,14.18,12.5,16.1,0.0,TRUE,21252
2015-06-25,14.72,12.1,17.4,1.0,FALSE,19463


### Usando vários verbos ao mesmo tempo

Vamos agora usar o `summarise()` junto com o `filter()` com as mesmas operações feitas antes, note a diferença. <br>
Agora, estamos fazendo as medidas resumo apenas para o que esta na base filtrada

In [34]:
dataset %>%
        filter(`Consumo de cerveja (litros)` > mean(`Consumo de cerveja (litros)`, na.rm = TRUE))%>%
        group_by(`Final de Semana`) %>%
        summarise(`Média de consumo de cerveja (litros)` = mean(`Consumo de cerveja (litros)`), 
                  `Desvio Padrão no consumo de cerveja (litros)` = sd(`Consumo de cerveja (litros)`))

Final de Semana,Média de consumo de cerveja (litros),Desvio Padrão no consumo de cerveja (litros)
<lgl>,<dbl>,<dbl>
FALSE,28313.24,2278.434
TRUE,30440.48,2781.788


### Importante!
É importante saber que muitas dessas operações não necessitam no `dplyr` para serem feitas, existindo funções nativas do `R` que dariam conta do recado.
<br> A grande facilidade do `dplyr` é que o código se torna extremamente legível.